# Importing all moldules

In [19]:
import alpha_vantage
import pandas
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import numpy as np

# CLASS ScriptData

In [20]:
class ScriptData(dict):

    global libray 
    libray = dict()

    def __getitem__(self,key):
        #print(type(lb[key]))
        #print(libray[key])
        return libray[key]


    def __setitem__(self, i, o):
        pass
    
    
    
    def __contains__(self, o):
        for key in libray.keys():
            if key == o:
                return True
            
        return False
    
    # __init__ function
    # def __init__(self):
    #     self = Userdict()


    def fetch_intraday_data(self,script):
        key = 'XTP4AMN2N5H9BL51'
        ts = TimeSeries(key,output_format='pandas')
        global data ,meta 
        data, meta = ts.get_intraday(script,interval='1min',outputsize='full')


    def convert_intraday_data(self,script):
        columns=['open','high','low','close','volumn']
        data.columns = columns
        data['timestamp']=data.index
        data.reset_index(drop=True, inplace=True)
        #data['timestamp']=data.index
        libray[script]=data
        #self['GOOGL'] = data
        #print(self[script])




    

In [21]:
script_data = ScriptData()

In [22]:
script_data.fetch_intraday_data('GOOGL')

In [23]:
script_data.convert_intraday_data('GOOGL')

In [24]:
script_data['GOOGL']

,open,high,low,close,volumn,timestamp
0,89.05,89.05,89.05,89.05,256.0,2022-12-23 20:00:00
1,89.12,89.12,89.11,89.11,375.0,2022-12-23 19:56:00
2,89.06,89.06,89.06,89.06,1127.0,2022-12-23 19:54:00
3,89.11,89.11,89.06,89.06,377.0,2022-12-23 19:48:00
4,89.09,89.09,89.09,89.09,140.0,2022-12-23 19:35:00
...,...,...,...,...,...,...
5963,92.90,92.91,92.90,92.91,549.0,2022-12-12 04:10:00
5964,92.95,92.95,92.95,92.95,416.0,2022-12-12 04:04:00
5965,92.93,92.94,92.93,92.94,427.0,2022-12-12 04:03:00
5966,92.96,92.96,92.90,92.90,1264.0,2022-12-12 04:02:00


In [25]:
script_data.fetch_intraday_data('AAPL')

In [26]:
script_data.convert_intraday_data('AAPL')

In [27]:
script_data['AAPL']

,open,high,low,close,volumn,timestamp
0,131.6500,131.6500,131.6500,131.6500,648.0,2022-12-23 20:00:00
1,131.7300,131.7300,131.7000,131.7000,1281.0,2022-12-23 19:58:00
2,131.7499,131.7499,131.7100,131.7300,3295.0,2022-12-23 19:57:00
3,131.7400,131.7400,131.7400,131.7400,581.0,2022-12-23 19:56:00
4,131.7400,131.7400,131.7400,131.7400,609.0,2022-12-23 19:55:00
...,...,...,...,...,...,...
7940,142.3001,142.3600,142.3001,142.3600,1985.0,2022-12-12 04:17:00
7941,142.1603,142.1902,142.1603,142.1902,396.0,2022-12-12 04:10:00
7942,142.1403,142.1403,142.1403,142.1403,216.0,2022-12-12 04:09:00
7943,142.1103,142.1203,142.1103,142.1203,840.0,2022-12-12 04:06:00


In [28]:
'GOOGL' in script_data

True

In [29]:
'AAPL' in script_data 

True

In [30]:
'UNCLE' in script_data

False

# Indicator method


In [31]:
def indicator1(df,timeperiod):
    df1 = pandas.DataFrame()
    df1['indicator'] = df['close'].rolling(timeperiod).mean()
    return df1

In [32]:
indicator1(script_data['GOOGL'], timeperiod=5)

,indicator
0,NaN
1,NaN
2,NaN
3,NaN
4,89.074
...,...
5963,92.894
5964,92.912
5965,92.934
5966,92.922


In [33]:
indicator1(script_data['AAPL'], 5)

,indicator
0,NaN
1,NaN
2,NaN
3,NaN
4,131.71200
...,...
7940,142.31208
7941,142.30408
7942,142.27412
7943,142.22818


In [34]:

class Strategy(ScriptData):
    
    def __init__(self,script):
        self.script = script


    def get_script_data(self):
        self.fetch_intraday_data(self.script)
        self.convert_intraday_data(self.script)
        global df
        global indicator_data
        df = data['close'].to_frame()
        df['timestamp'] = data['timestamp']
        indicator_data = indicator1(df, timeperiod=5)
        print(indicator_data.head())
        

    def get_signals(self):
        
        signals = df['timestamp'].to_frame()
        a = []
        b = []
        count = 0
        for c,d in zip(df['close'] , indicator_data['indicator']):
            a.append(c)
            b.append(d)
            count+=1
        array1 = np.stack((a,b), axis=1)
        
        #print(array1)
        #print(array1[0][1])
        signal= []
        for i in range(count):
            
            if(float(array1[i][0])==float(array1[i][1])):
                    #signal.append('SELL')
                    
                    if(array1[i-1][1]>array1[i-1][0]):
                        '''cut downwards = SELL'''
                        signal.append('SELL')
                    elif(array1[i-1][1]<array1[i-1][0]):
                        '''cut upwards = BUY'''
                        signal.append('BUY')
            else:
                '''no signal'''
                signal.append('NO_SIGNAL')
        #print(signal)       
        signals['signal']=pandas.Series(signal)   
        print(signals)
        
        
        
        
        
        
         
            
            
        
        
        
        #df2 = df['timestamp'].to_frame()
         #df2['df_close'] = df['close']
         #df2['indicator_data']= indicator_data['indicator']
         #plt.plot(df2['timestamp'],df2['df_close'],color='red')
         #plt.plot(df2['timestamp'],df2['indicator_data'],color='navy')
        # stratergy_data[''] = df[]


In [35]:
strategy = Strategy('NVDA')
strategy.get_script_data()

   indicator
0        NaN
1        NaN
2        NaN
3        NaN
4     151.83


In [36]:
strategy.get_signals()

               timestamp     signal
0    2022-12-23 19:59:00  NO_SIGNAL
1    2022-12-23 19:57:00  NO_SIGNAL
2    2022-12-23 19:55:00  NO_SIGNAL
3    2022-12-23 19:52:00  NO_SIGNAL
4    2022-12-23 19:48:00  NO_SIGNAL
...                  ...        ...
6795 2022-12-12 04:07:00  NO_SIGNAL
6796 2022-12-12 04:06:00  NO_SIGNAL
6797 2022-12-12 04:05:00        NaN
6798 2022-12-12 04:04:00        NaN
6799 2022-12-12 04:03:00        NaN

[6800 rows x 2 columns]
